Utilizando las herramientas que proveen distintas bibliotecas de Python (Numpy, Sklearn, etc.), se quiere realizar un análisis sobre reducción de dimensiones y visualización de datos. Se utilizará un set de datos sobre el cual se puede analizar la relación entre distintas muestras de tejido en términos de expresión genética.

El primer archivo contiene 1641 observaciones, y el segundo archivo contiene para cada una de esas observaciones su label correspondiente respetando el orden de los registros. Es decir, por ejemplo, el label del primer registro de GTExdata.csv es el primer registro del archivo SampleLabels.csv y así sucesivamente. El label que vamos a usar es Tissue type.

Se plantean los siguientes puntos a realizar:

1) Realizar una descomposición en valores singulares sobre el set y luego:

* Graficar la energía acumulada en función de la cantidad de autovalores, y graficar los autovalores del set.

* Con los dos gráficos anteriores, indicar qué valor de k se podría utilizar para realizar una aproximación. 

* Realizar una reducción a dos dimensiones del set de datos y graficar los puntos en un scatter-plot utilizando colores para indicar el label correspondiente a cada punto.

2) Utilizando el algoritmo t-SNE, se pide:

* Realizar un scatter-plot en el plano 2D, utilizando colores para indicar el label.

* Probar varios valores de perplexity (e.g. 3, 30, 1000). ¿Qué efectos tiene el perplexity en el gráfico?


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.linalg import svd
from sklearn.manifold import TSNE

In [ ]:
sns.set(style="darkgrid")
plt.rcParams['figure.figsize'] = (25,15)

In [ ]:
df_gtex = pd.read_csv('GTExdata.csv',low_memory=False)
display(df_gtex.head())

df_sample = pd.read_csv('SampleLabels.csv',low_memory=False)
display(df_sample.head())

# Solo nos importa 'Tissue Type'
df_sample = df_sample[['Tissue type']]

---

1) Realizar una descomposición en valores singulares sobre el set y luego:

* Graficar la energía acumulada en función de la cantidad de autovalores, y graficar los autovalores del set.

* Con los dos gráficos anteriores, indicar qué valor de k se podría utilizar para realizar una aproximación. 

* Realizar una reducción a dos dimensiones del set de datos y graficar los puntos en un scatter-plot utilizando colores para indicar el label correspondiente a cada punto.

In [ ]:
u, s, vt = svd(df_gtex)
display(u[:3],s[:3],vt[:3])

In [ ]:
energia_total = sum(s**2)
autovalores_al_cuadrado = s**2
energia_acumulada = np.cumsum(autovalores_al_cuadrado)

energia_acumulada[:3]

In [ ]:
plt.subplot(1,2,1)
plt.plot(energia_acumulada)
plt.title('Energía acumulada en función de la cantidad de autovalores')
plt.xlabel('Cantidad de autovalores')
plt.ylabel('Energía acumulada')

plt.subplot(1,2,2)
plt.plot(s)
plt.title('Autovalores')
plt.xlabel('Nro de autovalor')
plt.ylabel('Valor')

Podemos ver que los primeros 250 autovalores son los que contienen la mayor parte de la información, o incluso menos que esto. Busquemos el codo dentro de estos

In [ ]:
porcentaje = ((autovalores_al_cuadrado / energia_total) * 100)
porcentaje_acumulado = np.cumsum(porcentaje)

primeros_250 = porcentaje_acumulado[:250]
plt.plot(primeros_250)

plt.title('Energía acumulada en función de la cantidad de autovalores para los primeros 250 autovalores')
plt.xlabel('Cantidad de autovalores')
plt.ylabel('Energía acumulada en porcentaje')

Hacemos un zoom mas

In [ ]:
primeros_50 = porcentaje_acumulado[:50]
plt.plot(primeros_50)

plt.title('Energía acumulada en función de la cantidad de autovalores para los primeros 50 autovalores')
plt.xlabel('Cantidad de autovalores')
plt.ylabel('Energía acumulada en porcentaje')

In [ ]:
print('Con k=10 se preserva un {:.2f}% de la información'.format(porcentaje_acumulado[10]))
print('Con k=20 se preserva un {:.2f}% de la información'.format(porcentaje_acumulado[20]))
print('Con k=30 se preserva un {:.2f}% de la información'.format(porcentaje_acumulado[30]))
print('Con k=40 se preserva un {:.2f}% de la información'.format(porcentaje_acumulado[40]))
print('Con k=50 se preserva un {:.2f}% de la información'.format(porcentaje_acumulado[50]))

Se concluye que se puede usars un K entre 40 y 50.

In [ ]:
# Realizo el producto interno entre U y Sigma y sobre el resultado me quedo con las dos primeras columnas

reduccion_k_2 = u * s
reduccion_k_2 = np.matrix(reduccion_k_2)[:,:2]
reduccion_k_2 = pd.DataFrame(reduccion_k_2)
reduccion_k_2 = reduccion_k_2.merge(df_sample, left_index=True, right_index=True)
reduccion_k_2.head()

In [ ]:
sns.scatterplot(0,1,data=reduccion_k_2,hue='Tissue type')
plt.title('Información reducida a 2 dimensiones')

---

2) Utilizando el algoritmo t-SNE, se pide:

* Realizar un scatter-plot en el plano 2D, utilizando colores para indicar el label.

* Probar varios valores de perplexity (e.g. 3, 30, 1000). ¿Qué efectos tiene el perplexity en el gráfico?

In [ ]:
tsne = TSNE(n_components=2, verbose=1)
tsne_resultado = tsne.fit_transform(df_gtex)
df_tsne = pd.DataFrame(tsne_resultado)
df_tsne = df_tsne.merge(df_sample, left_index=True, right_index=True)

sns.scatterplot(x=0,y=1,data=df_tsne,hue='Tissue type')
plt.title('TSNE con todos los datos')

In [ ]:
perplexities = [1,10,50]
seleccion = 500

fig, axs = plt.subplots(ncols=len(perplexities))

for i,p in enumerate(perplexities):
    tsne = TSNE(n_components=2,perplexity=p)
    tsne_resultado = tsne.fit_transform(df_gtex[:seleccion])
    df_tsne = pd.DataFrame(tsne_resultado)
    df_tsne = df_tsne.merge(df_sample, left_index=True, right_index=True)
    sns.scatterplot(x=0,y=1,data=df_tsne,hue='Tissue type',ax=axs[i])
    axs[i].set_title('TSNE con {} registros, Perplexity={}'.format(seleccion, p))

In [ ]:
perplexities = [100,1000,10000]
seleccion = 500

fig, axs = plt.subplots(ncols=len(perplexities))

for i,p in enumerate(perplexities):
    tsne = TSNE(n_components=2,perplexity=p)
    tsne_resultado = tsne.fit_transform(df_gtex[:seleccion])
    df_tsne = pd.DataFrame(tsne_resultado)
    df_tsne = df_tsne.merge(df_sample, left_index=True, right_index=True)
    sns.scatterplot(x=0,y=1,data=df_tsne,hue='Tissue type',ax=axs[i])
    axs[i].set_title('TSNE con {} registros, Perplexity={}'.format(seleccion, p))

Se puede ver como la variable de la perplejidad modifica como se agrupan los datos. A menor perplejidad, más grupos se formaran, aislandose del resto, mientras que a mayor todo será parte de la misma gran nube.